<a href="https://colab.research.google.com/github/ranianasser/ass.4/blob/main/MK_Plant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications.mobilenet import MobileNet
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
batch_size = 32
img_size=224

train_generator=ImageDataGenerator(
    rescale=1./255
).flow_from_directory(
        '/content/drive/MyDrive/MK/train',
target_size = (img_size, img_size),batch_size = batch_size)

test_generator=ImageDataGenerator(
    rescale=1./255
).flow_from_directory(
    '/content/drive/MyDrive/MK/test',
target_size = (img_size, img_size),batch_size = batch_size)

Found 104 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
imgs,labels=next(train_generator)

In [5]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(imgs, labels, test_size=0.15, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(27, 224, 224, 3)
(27, 2)
(5, 224, 224, 3)
(5, 2)


In [6]:
from keras.applications.mobilenet import MobileNet
mobile=MobileNet()
model=Sequential()
for layer in mobile.layers[:-1]:
  model.add(layer)

for layer in model.layers:
  layer.trainable = False # Not trainable weights

model.add(Dense(2, activation='softmax'))
model.summary()

17225924/17225924 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128       
 zation)                                                 

In [7]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [8]:
# Create an ImageDataGenerator object
data_augmentation = ImageDataGenerator(
    rotation_range=20,  #randomly rotated by up to 20 degrees in either direction.
    width_shift_range=0.3,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.3,  # randomly shift images vertically (fraction of total height)
    zoom_range=0.3,  # set range for random zoom
    horizontal_flip=True,  # randomly flip images
)

train_data_augmentation = data_augmentation.flow(X_train, y_train)
validation_data_augmentation = data_augmentation.flow(X_val, y_val)

In [9]:
# Fit Model
early_stopping_monitor = EarlyStopping(patience = 2)
model.fit(X_train,y_train, validation_data=validation_data_augmentation, epochs = 3, callbacks = [early_stopping_monitor])

Epoch 1/3
1/1 [==============================] - 5s 5s/step - loss: 2.7077 - accuracy: 0.5556 - val_loss: 0.0426 - val_accuracy: 1.0000
Epoch 2/3
1/1 [==============================] - 4s 4s/step - loss: 1.2536 - accuracy: 0.7407 - val_loss: 0.4763 - val_accuracy: 0.6000
Epoch 3/3
1/1 [==============================] - 3s 3s/step - loss: 0.6219 - accuracy: 0.7778 - val_loss: 1.7623 - val_accuracy: 0.4000


In [10]:
test_labels=test_generator.classes
predictions=np.argmax(model.predict(test_generator),axis=1)
print(predictions[:5])

1/1 [==============================] - 4s 4s/step
[0 1 0 0 0]


In [11]:
confusion_matrix(test_labels,predictions)

array([[10,  2],
       [11,  1]])

In [13]:
# Evaluate the model
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.4583333333333333
Precision: 0.3333333333333333
Recall: 0.08333333333333333
F1-score: 0.13333333333333333
